# Functionality:  visualization of graph

## Import Packages

In [2]:
import igraph as ig
import igraph.drawing.colors as igcolors
from igraph.drawing.text import TextDrawer
import cairo

import numpy as np

## Random Matrix

In [3]:
def randweight_sysmetric(nchns=64):
    w = np.random.rand(nchns, nchns)
    
    weight = (w + w.T)/2
    
    return weight

## Create Graph

### generate a graph from its adjacency matrix

In [4]:
def graph_create(weight, tagdirected = False, vsarea = None, vscoord = None, vschni = None):
    """
        Create graph based on weight matrix
        
        Args:
            
            weight: weight matrix (should be symmetrics if indirected graph)
            
            tagdirected: tag for if directed (True) or indirected (False, default)
            
            vsarea: a list of string representing vertex area
            
        return:
            graph: the generated graph
            
    """

    #  adjacency matrix
    adjmatrix = (weight>0).tolist()

    # create graph from adjacency matrix
    if not tagdirected:
        """ undirected graph """

        # check the symmetric of weights matrix
        if not np.allclose(weight, weight.T, rtol=1e-05, atol=1e-08):
            print("weight matrix is not symmetric")
            return []
        
        
        # generate a undirected graph from its adjacency matrix (weights>0)
        graph = ig.Graph.Adjacency(adjmatrix, "UNDIRECTED")
        


        # weights for the undirected graph, lower triangular 
        wght = np.triu(weight)

    else:
        """ directed graph """

        # generate a undirected graph from its adjacency matrix (weights>0)
        graph = ig.Graph.Adjacency(adjmatrix, "DIRECTED")

        # weights for the directed graph
        wght = weight
        
    
    
    
    # set the vs attributes area
    if vsarea is not None:
        graph.vs["area"] = vsarea
    
    # set the vs position
    if vscoord is not None:
        graph.vs["coord"] = vscoord
    
    if vschni is not None:
        graph.vs['name'] = vschni
    
    # set the attribute of weight for graph element es
    graph.es["weight"] = wght[wght.nonzero()]
    
    return graph

## Graph Visulization

### Set visulization style

In [5]:
def graph_style(graph):
    """
    
        args:
            graph: graph
            
        return:
            visual_style: a dictionary containing the visualization style
    """

    visual_style = dict()

    # vertex color
    try:
        "set vertex  color based on vs attribute area"
        
        # unique areas
        uniqarea = list(set(graph.vs["area"]))
        
        # set palette
        palette = igcolors.ClusterColoringPalette(len(uniqarea))
        
        # set vertex color
        visual_style['vertex_color'] = [palette[uniqarea.index(area)] for area in graph.vs["area"]]
    
    except KeyError:
        
        visual_style['vertex_color'] = "black"
        
    
    # vertex size
    outdegree = graph.outdegree()
    if max(outdegree) > 0:
        visual_style["vertex_size"] = [x/max(outdegree)*10+5 for x in outdegree]
    
    # vertex label size
    visual_style['vertex_label_size'] = 2

    # vertex label distance
    visual_style['vertex_label_dist'] = 2

    # vertex label color
    visual_style['vertex_label_color'] = 'black'
    
    # vertex label name
    visual_style["vertex_label"] = graph.vs['name']
    
    visual_style["vertex_label_size"] = 10

    # the outdegree for each vertex
    outdegree = graph.outdegree()

    

    # edge width
    visual_style['edge_width'] = graph.es['weight']

    # layout
    try:
        "set vertex  layout based on vs attribute coordinate coord"
        visual_style['layout'] = ig.Layout(graph.vs["coord"])
    
    except KeyError:
        print("Does not have a user defined layout, use the default.")
    
    return visual_style

### Actual Plot

In [14]:
def graph_plot(graph, visual_style, texts = None):
    """
        @param graph:
        
        @param visual_style:
        
        @param texts: a dictionary storing the text to be printed with text[key]: [x,y, fontsize]
        e.g texts["M1"] = [0, 180, 30]
        
        @return igplot: an igraph.drawing.Plot object
    """
    
    # an igraph.drawing.Plot object
    igplot = ig.plot(graph, **visual_style)
    
    # plot text
    if texts is not None:
        
        igplot.redraw()
            
        # Context object
        ctx = cairo.Context(igplot.surface)
        
        
        # draw each text in ig.plot
        for text in texts:
            val = texts[text]
            
            # fontsize
            try:
                fontsize = val[2]
            
            except IndexError:
                fontsize = 20
            
            
            # set the font size
            ctx.set_font_size(fontsize)
            
            # TextDrawer Object
            drawer = TextDrawer(ctx, text, halign=TextDrawer.CENTER)
            
            # draw the text at the coordinates coord
            drawer.draw_at(x=val[0], y=val[1], width=300)
            
    return igplot

## Test Section

In [7]:
weight = randweight_sysmetric(nchns=15)
graph = graph_create(weight)